## Import libraried and read data

In [1]:
import sys
from google.colab import drive
drive.mount('/content/gdrive')
colab_dir = '/content/gdrive/My Drive/Colab_files/2022_CCS_case_study'
sys.path.append(colab_dir)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install pmdarima
from pmdarima.arima import auto_arima

In [3]:
%matplotlib inline

from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
plt.style.use('seaborn')
import pandas as pd
import numpy as np

In [16]:
import warnings
warnings.filterwarnings('ignore')
#!pip install plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
#from sklearn.model_selection import GridSearchCV
#from sklearn.preprocessing import StandardScaler
#from sklearn.cluster import KMeans
#from sklearn.metrics import silhouette_score,silhouette_samples
#from sklearn.linear_model import LinearRegression,Ridge,Lasso
#from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score
#import statsmodels.api as sm
#from statsmodels.tsa.api import Holt,SimpleExpSmoothing,ExponentialSmoothing
from fbprophet import Prophet
#from sklearn.preprocessing import PolynomialFeatures
#from statsmodels.tsa.stattools import adfuller
#!pip install pyramid-arima
#from pyramid.arima import auto_arima
#std=StandardScaler()

In [4]:
df = pd.read_csv(colab_dir+'/canada_aggregation_divisions.csv',  index_col=0)
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
#df = df.set_index('date')
print(df.size)
df.head(2)

62469


,date,PRNAME,CDUID,conn_type,avg_d_mbps_wt,avg_u_mbps_wt,avg_lat_ms,tests,DA_POP
0,2019-01-01,Alberta,4801,fixed,51.526124,8.122863,41.877760,1268,246025.0
1,2019-01-01,Alberta,4801,mobile,28.724486,7.500324,41.990991,111,17665.0


In [5]:
df.describe()

,CDUID,avg_d_mbps_wt,avg_u_mbps_wt,avg_lat_ms,tests,DA_POP
count,6941.000000,6941.000000,6941.000000,6941.000000,6941.000000,6.941000e+03
mean,3331.826106,55.124235,18.366608,52.554231,1204.322576,1.473459e+05
std,1503.528828,35.061046,19.768781,33.572339,2230.627052,2.629336e+05
min,1001.000000,0.264000,0.380833,5.000000,1.000000,0.000000e+00
25%,2427.000000,28.959806,6.516930,34.688889,64.000000,1.856000e+04
50%,3501.000000,47.070699,10.956312,46.260870,333.000000,6.169000e+04
75%,4701.000000,72.982751,21.867103,61.779102,1321.000000,1.617750e+05
max,6208.000000,235.600000,201.910572,693.000000,26380.000000,3.891230e+06


In [6]:
def get_stat(in_df, in_group_list):
  county_stats = (
    in_df.groupby(in_group_list)
    .apply(
        lambda x: pd.Series(
            {"avg_d_mbps": np.average(x["avg_d_mbps_wt"], weights=x["tests"]),
             "avg_u_mbps": np.average(x["avg_u_mbps_wt"], weights=x["tests"]),
             "avg_lat_ms": np.average(x["avg_lat_ms"], weights=x["tests"])}
        )
    )
    .reset_index()
    .merge(
        in_df.groupby(in_group_list)
        .agg(tests=("tests", "sum"), DA_POP=("DA_POP", "sum"))
        .reset_index(),
        on=in_group_list,
    )
  )
  return county_stats

tests_stat = get_stat(df, ['date','PRNAME', 'CDUID'])
tests_stat.head()

,date,PRNAME,CDUID,avg_d_mbps,avg_u_mbps,avg_lat_ms,tests,DA_POP
0,2019-01-01,Alberta,4801,49.690749,8.072753,41.886875,1379,263690.0
1,2019-01-01,Alberta,4802,72.662193,23.299422,32.057443,2472,596265.0
2,2019-01-01,Alberta,4803,35.584714,14.525885,42.413421,1669,242090.0
3,2019-01-01,Alberta,4804,14.772167,7.301498,66.660714,448,50455.0
4,2019-01-01,Alberta,4805,25.067375,7.254570,42.145426,2558,300675.0


## Make a prediction for a single Census devision with CDUID=4801

In [7]:
df_ftr = tests_stat[tests_stat.CDUID==4801]
df_ftr = df_ftr.drop(columns=['PRNAME', 'CDUID'])
df_ftr

,date,avg_d_mbps,avg_u_mbps,avg_lat_ms,tests,DA_POP
0,2019-01-01,49.690749,8.072753,41.886875,1379,263690.0
291,2019-04-01,63.245842,8.758141,56.660928,1336,243150.0
581,2019-07-01,59.620876,7.540486,56.803340,1078,239865.0
871,2019-10-01,63.759430,7.666982,39.650307,1304,266555.0
1161,2020-01-01,53.747842,7.077155,39.555630,1483,274445.0
1452,2020-04-01,47.353350,7.739009,44.363132,1622,260995.0
1743,2020-07-01,59.037238,19.380561,38.209581,1837,300015.0
2033,2020-10-01,45.983139,12.187001,45.211782,1341,242955.0
2325,2021-01-01,50.058416,11.773874,37.569746,1613,293085.0
2616,2021-04-01,82.909560,16.414656,31.818584,1582,304500.0


In [8]:
model_train=df_ftr.iloc[:int(df_ftr.shape[0]-1)]
valid=df_ftr.iloc[int(df_ftr.shape[0]-1):]
y_pred=valid.copy()
print(y_pred)

           date  avg_d_mbps  avg_u_mbps  avg_lat_ms  tests    DA_POP
3198 2021-10-01   66.359073   16.415049   36.057464   1601  286590.0


## AR Model (using AUTO ARIMA)

In [58]:
# https://www.kaggle.com/code/neelkudu28/covid-19-visualizations-predictions-forecasting#Time-Series-Forecasting
# AR Model (using AUTO ARIMA)
model_ar= auto_arima(model_train["avg_d_mbps"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=4,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_ar.fit(model_train["avg_d_mbps"])

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=122.849, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=97.043, Time=0.01 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=97.140, Time=0.03 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=inf, Time=0.04 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0]          
Total fit time: 0.099 seconds


ARIMA(order=(1, 0, 0), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [59]:
prediction_ar=model_ar.predict(len(valid))
y_pred["AR Model Prediction"]=prediction_ar

In [60]:
model_scores=[]
model_scores.append(np.sqrt(mean_squared_error(y_pred["avg_d_mbps"],y_pred["AR Model Prediction"])))
print("Root Mean Square Error for AR Model: ",np.sqrt(mean_squared_error(y_pred["avg_d_mbps"],y_pred["AR Model Prediction"])))

Root Mean Square Error for AR Model:  6.531194237847565


In [61]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train["avg_d_mbps"],
                    mode='lines+markers',name="Train Data for download speed"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["avg_d_mbps"],
                    mode='lines+markers',name="Validation Data for for download speed",))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["AR Model Prediction"],
                    mode='lines+markers',name="Prediction of for download speed",))
fig.update_layout(title="Download speed AR Model Prediction",
                 xaxis_title="Quaters",yaxis_title="Download speed",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

# ARIMA Model (using AUTOARIMA)

In [62]:
model_arima= auto_arima(model_train["avg_d_mbps"],trace=True, error_action='ignore', start_p=1,start_q=1,max_p=3,max_q=3,
                   suppress_warnings=True,stepwise=False,seasonal=False)
model_arima.fit(model_train["avg_d_mbps"])

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=122.849, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[0]             : AIC=inf, Time=0.04 sec
 ARIMA(0,0,2)(0,0,0)[0]             : AIC=inf, Time=0.07 sec
 ARIMA(0,0,3)(0,0,0)[0]             : AIC=nan, Time=0.05 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=97.043, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[0]             : AIC=inf, Time=0.10 sec
 ARIMA(1,0,2)(0,0,0)[0]             : AIC=inf, Time=0.11 sec
 ARIMA(1,0,3)(0,0,0)[0]             : AIC=inf, Time=nan sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=97.140, Time=0.04 sec
 ARIMA(2,0,1)(0,0,0)[0]             : AIC=96.772, Time=0.13 sec
 ARIMA(2,0,2)(0,0,0)[0]             : AIC=inf, Time=0.20 sec
 ARIMA(2,0,3)(0,0,0)[0]             : AIC=inf, Time=nan sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=inf, Time=0.05 sec
 ARIMA(3,0,1)(0,0,0)[0]             : AIC=inf, Time=0.12 sec
 ARIMA(3,0,2)(0,0,0)[0]             : AIC=97.923, Time=0.21 sec

Best model:  ARIMA(2,0,1)(0,0,0)[0]          
Total fit time: 1.587 se

ARIMA(order=(2, 0, 1), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [63]:
prediction_arima=model_arima.predict(len(valid))
y_pred["ARIMA Model Prediction"]=prediction_arima

In [64]:
model_scores.append(np.sqrt(mean_squared_error(valid["avg_d_mbps"],prediction_arima)))
print("Root Mean Square Error for ARIMA Model: ",np.sqrt(mean_squared_error(valid["avg_d_mbps"],prediction_arima)))

Root Mean Square Error for ARIMA Model:  8.354460161478627


# AR Model with exogenous variables

In [65]:
exogenous_var = ['DA_POP']
model_ar= auto_arima(model_train["avg_d_mbps"],trace=True, error_action='ignore', start_p=0,start_q=0,max_p=4,max_q=0,
                   suppress_warnings=True,stepwise=False,seasonal=False,
                   exogeneous=model_train[exogenous_var], stationarity=False, seasonality=True)
model_ar.fit(model_train["avg_d_mbps"], exogenous=model_train[exogenous_var])

 ARIMA(0,0,0)(0,0,0)[0]             : AIC=122.849, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[0]             : AIC=97.043, Time=0.01 sec
 ARIMA(2,0,0)(0,0,0)[0]             : AIC=97.140, Time=0.03 sec
 ARIMA(3,0,0)(0,0,0)[0]             : AIC=inf, Time=0.05 sec

Best model:  ARIMA(1,0,0)(0,0,0)[0]          
Total fit time: 0.114 seconds


ARIMA(order=(1, 0, 0), scoring_args={}, suppress_warnings=True,
      with_intercept=False)

In [66]:
prediction_ar=model_ar.predict(len(valid), exogenous=valid[exogenous_var])
y_pred["AR Model Prediction with exogenous variables"]=prediction_ar

In [67]:
model_scores.append(np.sqrt(mean_squared_error(y_pred["avg_d_mbps"],y_pred["AR Model Prediction with exogenous variables"])))
print("Root Mean Square Error for AR Model: ",np.sqrt(mean_squared_error(y_pred["avg_d_mbps"],y_pred["AR Model Prediction with exogenous variables"])))

Root Mean Square Error for AR Model:  5.294172542859052


In [68]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=model_train.index, y=model_train["avg_d_mbps"],
                    mode='lines+markers',name="Train Data for download speed"))
fig.add_trace(go.Scatter(x=valid.index, y=valid["avg_d_mbps"],
                    mode='lines+markers',name="Validation Data for for download speed",))
fig.add_trace(go.Scatter(x=valid.index, y=y_pred["AR Model Prediction with exogenous variables"],
                    mode='lines+markers',name="Prediction of for download speed",))
fig.update_layout(title="Download speed AR Model Prediction with exogenous variables",
                 xaxis_title="Quaters",yaxis_title="Download speed",legend=dict(x=0,y=1,traceorder="normal"))
fig.show()

# Model comparison

In [70]:
model_names=["Auto Regressive Model (AR)","ARIMA Model","Auto Regressive Model with exogenous variables"]
model_summary=pd.DataFrame(zip(model_names,model_scores),columns=["Model Name","Root Mean Squared Error"]).sort_values(["Root Mean Squared Error"])
model_summary

,Model Name,Root Mean Squared Error
2,Auto Regressive Model with exogenous variables,5.294173
0,Auto Regressive Model (AR),6.531194
1,ARIMA Model,8.354460
